In [428]:
import pandas as pd
from pandas.api.types import CategoricalDtype
df = pd.read_csv('fatal-police-shootings-data.csv')

In [429]:
df_grouped=df.groupby(['race'],as_index=False).size()
df_mental=df.groupby(['race'])['signs_of_mental_illness'].sum()
df_grouped=df_grouped.join(df_mental,on='race')
df_grouped.rename(columns={"size":"race_count"},inplace=True)
df_mental=df_mental.to_frame()
df_grouped

,race,race_count,signs_of_mental_illness
0,A,93,24
1,B,1298,184
2,H,902,157
3,N,78,14
4,O,48,11
5,W,2476,713


In [430]:
def rate(row):

    percentage = (row['signs_of_mental_illness'] /row['race_count'] )
    return round(percentage*100,2)

In [431]:
df_grouped['rate'] = df_grouped.apply(lambda row: rate(row),axis=1)
df_grouped

,race,race_count,signs_of_mental_illness,rate
0,A,93,24,25.81
1,B,1298,184,14.18
2,H,902,157,17.41
3,N,78,14,17.95
4,O,48,11,22.92
5,W,2476,713,28.80


Największy odsetek z rasy W

In [432]:
df['date'] = pd.to_datetime(df['date'])
df_converted=df['date'].dt.day_name()
df.insert(3,"day", df_converted)

In [433]:
df

,id,name,date,day,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2015-01-02,Friday,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2015-01-02,Friday,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,2015-01-03,Saturday,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,2015-01-04,Sunday,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,2015-01-04,Sunday,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5411,5921,William Slyter,2020-06-13,Saturday,shot,gun,22.0,M,W,Kansas City,MO,False,other,Other,False
5412,5922,TK TK,2020-06-13,Saturday,shot,undetermined,NaN,M,NaN,San Bernardino,CA,False,attack,Not fleeing,False
5413,5924,Nicholas Hirsh,2020-06-15,Monday,shot,gun,31.0,M,W,Lawrence,KS,False,attack,Car,False
5414,5926,TK TK,2020-06-16,Tuesday,shot,gun,24.0,M,NaN,Beach Park,IL,False,attack,Not fleeing,False


In [434]:
order = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_day_grouped=df.groupby('day',as_index=False).size()
df_sorted = df_day_grouped.groupby(['day']).sum().reindex(order)

In [435]:
df_sorted.rename(columns={"size":"day_count"},inplace=True) 
df_sorted

,day_count
day,
Monday,726
Tuesday,816
Wednesday,829
Thursday,810
Friday,761
Saturday,721
Sunday,753


In [436]:
writer = pd.ExcelWriter('add_new_chart.xlsx', engine='xlsxwriter')
df_sorted.to_excel(writer, sheet_name='my_chart')

In [437]:
workbook = writer.book
worksheet = writer.sheets['my_chart']

In [438]:
chart = workbook.add_chart({'type':'column'})

In [439]:
def grab_series(df, sheet_name, colname, startcol=0, startrow=0):

    col_index = df.columns.tolist().index(colname)
    col_letter = chr(ord('@')+(col_index+2+startcol))
    first_row = startrow + 2
    last_row = startrow + 1 + len(df)
    return f"='{sheet_name}'!{col_letter}{first_row}:{col_letter}{last_row}"

In [440]:
chart.add_series({'values':grab_series(df_sorted,'my_chart','day_count')})

In [441]:
chart.set_x_axis({
    'name': 'day',
    'name_font': {'size': 14, 'bold': True},
    'num_font':  {'italic': True },
})

In [442]:
chart.set_legend({'none': True})
worksheet.insert_chart('F2', chart)

0

In [443]:
workbook.close()

In [444]:
df_2=pd.read_excel('population.xlsx')

In [445]:
df_3=pd.read_excel('abbreviation.xlsx')

In [446]:
df_3.loc[:, "population"] = df_2['population']
df_3=df_3.set_index('state')
df_joined=df.join(df_3,on='state')
df_joined=df_joined.groupby(['state','state_full_name','population']).size().to_frame('occurences')

In [447]:
def per_thousand(row):

    percentage = (row['occurences'] /row['population'] )
    return round(percentage*1000,3)

df_joined=df_joined.reset_index()

df_joined['per_thousand'] = df_joined.apply(lambda row: per_thousand(row),axis=1)

df_joined

,state,state_full_name,population,occurences,per_thousand
0,AK,Alaska,733391,39,0.053
1,AL,Alabama,5024279,104,0.021
2,AR,Arkansas,3011524,83,0.028
3,AZ,Arizona,7151502,254,0.036
4,CA,California,39538223,799,0.020
5,CO,Colorado,5773714,195,0.034
6,CT,Connecticut,3605944,21,0.006
7,DC,District of Columbia,689545,13,0.019
8,DE,Delaware,989948,13,0.013
9,FL,Florida,21538187,350,0.016
